# Python idioms that are more widely used in the python community, such as vectorization instead of loops whenever possible.

In [8]:
import pandas as pd
import numpy as np

# to count the time
import timeit

df = pd.read_csv('data/census.csv')
df = df.set_index(['STNAME', 'CTYNAME'])
df.head(3)

SUMLEV  REGION  DIVISION  STATE  COUNTY  \
STNAME  CTYNAME                                                   
Alabama Alabama             40       3         6      1       0   
        Autauga County      50       3         6      1       1   
        Baldwin County      50       3         6      1       3   

                        CENSUS2010POP  ESTIMATESBASE2010  POPESTIMATE2010  \
STNAME  CTYNAME                                                             
Alabama Alabama               4779736            4780127          4785161   
        Autauga County          54571              54571            54660   
        Baldwin County         182265             182265           183193   

                        POPESTIMATE2011  POPESTIMATE2012  ...  \
STNAME  CTYNAME                                           ...   
Alabama Alabama                 4801108          4816089  ...   
        Autauga County            55253            55175  ...   
        Baldwin County           186659           190396  ...   

                        RDOMESTICMIG2011  RDOMESTICMIG2012  RDOMESTICMIG2013  \
STNAME  CTYNAME                                                                
Alabama Alabama                 0.002295         -0.193196          0.381066   
        Autauga County          7.242091         -2.915927         -3.012349   
        Baldwin County         14.832960         17.647293         21.845705   

                        RDOMESTICMIG2014  RDOMESTICMIG2015  RNETMIG2011  \
STNAME  CTYNAME                                                           
Alabama Alabama                 0.582002         -0.467369     1.030015   
        Autauga County          2.265971         -2.530799     7.606016   
        Baldwin County         19.243287         17.197872    15.844176   

                        RNETMIG2012  RNETMIG2013  RNETMIG2014  RNETMIG2015  
STNAME  CTYNAME                                                             
Alabama Alabama            0.826644     1.383282     1.724718     0.712594  
        Autauga County    -2.626146    -2.722002     2.592270    -2.187333  
        Baldwin County    18.559627    22.727626    20.317142    18.293499  

[3 rows x 98 columns]

### Method Chaining:
The general idea behind method chaining is that every method on an object returns a reference to that object.
* AS a result you can combine many methods into a line or one statement of code.

In [35]:
df = df.where(df['SUMLEV'] == 50).dropna().rename(columns={'ESTIMATESBASE2010': 'Estimate Base 2010'})

### Mapping & pandas (apply & applymap functions)

* 'Map' function is used to compare the values between two lists of same size or will combare until the end of one list (this is possible with any iterable object) 

* Pandas has a similar function known as 'applymap' which is used to compare the columns. In applymap you have provide a function which should operate on 'each cell in a dataframe' and then returns set is a DataFrame.

* 'apply' function is pandas is used to map across all of the rows in DataFrame.

In [47]:
def min_max(row):
    data = row[['POPESTIMATE2010',
               'POPESTIMATE2011',
               'POPESTIMATE2012',
               'POPESTIMATE2013',
               'POPESTIMATE2014',
               'POPESTIMATE2015']]
    return pd.Series({'min': np.min(data), 'max': np.max(data)})
# apply function takes the function which contains the operations and the axis as the parameters

df.apply(min_max, axis = 1).head(3)

min       max
STNAME  CTYNAME                           
Alabama Autauga County   54660.0   55347.0
        Baldwin County  183193.0  203709.0
        Barbour County   26489.0   27341.0

In [54]:
# you can do the above function by added the min and max columns in the 'df' instead of creating a new Series object

def min_max_df(row):
    data = row[['POPESTIMATE2010',
               'POPESTIMATE2011',
               'POPESTIMATE2012',
               'POPESTIMATE2013',
               'POPESTIMATE2014',
               'POPESTIMATE2015']]
    # create a new entry
    row['MIN'] = np.min(data)
    row['MAX'] = np.max(data)
    return row

df = df.apply(min_max_df, axis = 1)
df['MIN'].head()

0     54660.0
1    183193.0
2     26489.0
3     22512.0
4     57373.0
Name: MIN, dtype: float64

### The 'apply' function is an extremely useful tool in the pandas toolkit. 'apply' function is rarely used with the big function definitions, and is instead called with 'lambdas'
Hence, it is important to know how to read lambdas

In [64]:
# following is a way to call 'apply' multiple times on a single line using lambda

row = ['POPESTIMATE2010','POPESTIMATE2011','POPESTIMATE2012','POPESTIMATE2013','POPESTIMATE2014',
           'POPESTIMATE2015']
df.apply(lambda x: np.min(x[row]), axis = 1).head()

# lambda above is just an unnamed function in python, it takes a single parameter x and returns a single value, which
# in this case is the maximum of all columns associated with row x

0     54660.0
1    183193.0
2     26489.0
3     22512.0
4     57373.0
dtype: float64